# SMPC - Simple Aggregation & Multiplication Algorithm 

This notebook serves as the initial integration of simple aggregation & multiplication algorithms between 3+ parties without revealing their secret values using open source protocols. This is achieved by splitting each value into multiple shares, each of which operate like a private key.

Important notes:
* each secret is split into 3 shares as default
* multiplication protocol only works via splitting into 2 shares for now
* honest-but-curious (or passive) security


## Preparations

In [ ]:
# import modules
import random as rd  # generating random numbers for distributions
import numpy as np  # handling vectors and matrices
from generate_prime import generate_prime_number  # code for generating large prime numbers quickly

## Encryption

> Encryption doesn't use floats or real numbers but happens in a mathematical space called [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) which is basically the integers between `0` and `Q-1`, where `Q` is prime and \"big enough\" so that the space can contain all the numbers that we use in our algorithms. In practice, given a value `x` integer, we do `x % Q` to fit in the ring. (That's why we avoid using number `x' > Q`).

In [ ]:
Q = generate_prime_number(50)
Q

In [ ]:
# enryption function
def encrypt(x, n_shares=3):
    shares = tuple(rd.randrange(0,Q) for _ in range(n_shares-1))
    return (*shares, ((x - sum(shares)) % Q))

## Decryption

In [ ]:
# decryption function
def decrypt(*shares):
    return sum(shares) % Q

In [ ]:
# example encryption
ex_shares = encrypt(6041)
ex_shares

In [ ]:
decrypt(*ex_shares)

## Aggregation Function

`x + y = (x0 + x1 + x2) + (y0 + y1 + y2)`

`x + y = (x0 + y0) + (x1 + y1) + (x2 + y2)`

In [ ]:
# aggregation algorithm
def add(x, y):
    # x & y have to have the same number of shares (= length)
    return [(x[i] + y[i]) % Q for i in range(len(x))]

### Example for adding 3 secrets

In [ ]:
# example
# set 3 secret values and encrypt each into 3 (default) shares
x = encrypt(6041)
y = encrypt(59)
z = encrypt(900)
# what do the shares look like
[x, y, z]

In [ ]:
# add the encrypted values
agg = add(add(x,y),z)
agg

In [ ]:
# and decrypt the aggregated shares
decrypt(*agg)

## Multiplication Function (WIP - only works with secrets split in 2 shares for now)

`x * y = (x0 + x1) * (y0 + y1)`

`x * y = (x0 * y0) + (x1 * y1) + (x0 * y1) + (x1 * y0)`

---

Masking:
* masks: s, t
* masked values: alpha, beta

---

`alpha = (x0 - s0) + (x1 - s1)`

`beta = (y0 - t0) + (y1 - t1)`

---

`z0 = st0 + (s0 * beta) + (alpha * t0) + (alpha * beta)`

`z1 = st1 + (s1 * beta) + (alpha * t1)`

---

Equivalency:
`z0 + z1 = x * y`

In [ ]:
# generate additional independent values/keys which have a multiplicative relationship
def crypto_generator():
    # create triple
    s = rd.randrange(0,Q)
    t = rd.randrange(0,Q)
    st = (s*t)%Q
    
    # create shares
    s0, s1 = encrypt(s, n_shares=2)
    t0, t1 = encrypt(t, n_shares=2)
    st0, st1 = encrypt(st, n_shares=2)
    
    return s0, s1, t0, t1, st0, st1

# multiplication algorithm
def mul(x,y):
    s0, s1, t0, t1, st0, st1 = crypto_generator()
    
    alpha = (x[0]-s0)+(x[1]-s1)
    beta = (y[0]-t0)+(y[1]-t1)
    
    z0 = st0 + (s0 * beta) + (alpha * t0) + (alpha * beta)
    z1 = st1 + (s1 * beta) + (alpha * t1)
    
    return z0%Q, z1%Q

### Example for multiplying 2 secrets

In [ ]:
x = encrypt(50, n_shares=2)
y = encrypt(12, n_shares=2)

In [ ]:
z = mul(x,y)

In [ ]:
z

In [ ]:
decrypt(*z)

## Creating Classes of Private and Public Values

In [ ]:
class PublicValue:
    
    def __init__(self, value):
        self.value = value

In [ ]:
class PrivateValue:

    def __init__(self, value, share0=None, share1=None, share2=None):
        if not value is None:
            share0, share1, share2 = encrypt(value)
        self.share0 = share0
        self.share1 = share1
        self.share2 = share2

    def decrypt(self):
        return PublicValue(decrypt(self.share0, self.share1, self.share2))
    
    def add(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            share0 = (x.share0 + y.element) % Q
            share1 =  x.share1
            share2 =  x.share2
            return PrivateValue(None, share0, share1, share2)
        if type(y) is PrivateValue:
            share0 = (x.share0 + y.share0) % Q
            share1 = (x.share1 + y.share1) % Q
            share2 = (x.share2 + y.share2) % Q
            return PrivateValue(None, share0, share1, share2)

In [ ]:
x = PrivateValue(5)
y = PrivateValue(12)

In [ ]:
z = x.add(y=y)

In [ ]:
decrypt(z.share0, z.share1, z.share2)